# P05 schedule

## Preamble

This section contains package imports and general settings for the notebook.

In [322]:
import pandas as pd
import datetime as dt
import calendar
# import dateutil as du
import numpy as np
from pathlib import Path
import os
import re
import fuzzywuzzy as fz

# replace n with the number of columns/rows you want to see completely
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 400)

## User input
- Year 
- Path to the dirctory which contains the following documents:
    - Petra 3 schedule provided by Oliver Seeck
    - DOOR scheduling support - available after the ranking was confirmed in DOOR

In [337]:
year = 2023

# directory which contains all necessary files to import,
# starting from "./desycloud"
dc_path = Path('desycloud/documents/' + str(year) + '/p05_schedule/')
# path to Petra 3 schedule
dc_p3sch_path = dc_path / 'Schedule2023_draft_Status2022.08.18_shifted_timing.xlsx'
# path to DOOR scheduling support
dc_proposals_path = dc_path / '2023_1/door.scheduling.support.xlsx'

# School holidays in Hamburg, Niedersachsen and Schleswig-Holstein
school_holidays = {"Hamburg":
                   {"Weihnachtferien": pd.date_range(start="2023-01-01", end="2023-01-06"),
                    "Winterferien": pd.date_range(start="2023-01-27", end="2023-01-27"),
                    "Osterferien": pd.date_range(start="2023-03-06", end="2023-03-17"),
                    "Pfingstferien": pd.date_range(start="2023-05-15", end="2023-05-19"),
                    "Sommerferien": pd.date_range(start="2023-07-13", end="2023-08-23"),
                    "Herbstferien": pd.date_range(start="2023-10-02", end="2023-10-27")},
                   "Niedersachsen":
                   {"Weihnachtferien": pd.date_range(start="2023-01-01", end="2023-01-06"),
                    "Winterferien": pd.date_range(start="2023-01-30", end="2023-01-31"),
                    "Osterferien": pd.date_range(start="2023-03-27", end="2023-04-11"),
                    "Pfingstferien": pd.date_range(start="2023-05-19", end="2023-05-30"),
                    "Sommerferien": pd.date_range(start="2023-07-06", end="2023-08-16"),
                    "Herbstferien1": pd.date_range(start="2023-10-02", end="2023-10-02"),
                    "Herbstferien2": pd.date_range(start="2023-10-16", end="2023-10-30")},
                   "Schleswig-Holstein":
                   {"Weihnachtferien":  pd.date_range(start="2023-01-01", end="2023-01-07"),
                    "Osterferien": pd.date_range(start="2023-04-06", end="2023-04-22"),
                    "Pfingstferien": pd.date_range(start="2023-05-19", end="2023-05-20"),
                    "Sommerferien": pd.date_range(start="2023-07-17", end="2023-08-26"),
                    "Herbstferien": pd.date_range(start="2023-10-16", end="2023-10-27")}
                   }

# Holidays in Hamburg
hh_holidays = {"Neujahr":  dt.date(2023, 1, 1),
               "Karfreitag": dt.date(2023, 4, 7),
               "Ostermontag": dt.date(2023, 4, 10),
               "Tag der Arbeit": dt.date(2023, 5, 1),
               "Christi Himmelfahrt": dt.date(2023, 5, 18),
               "Pingstmontag": dt.date(2023, 5, 29),
               "Tag der Deutschen Einheit": dt.date(2023, 10, 3),
               "Reformationstag": dt.date(2023, 10, 31),
               "1. Weihnachtsfeiertag": dt.date(2023, 12, 25),
               "2. Weihnachtsfeiertag": dt.date(2023, 12, 26)
               }

# Absences
absences = {"FW": [[dt.date(2023, 7, 1), dt.date(2023, 8, 1)],
                   [dt.date(2023, 9, 1), dt.date(2023, 8, 1)]
                  ],
            "JH": [[dt.date(2023, 7, 1), dt.date(2023, 8, 1)],
                   [dt.date(2023, 9, 1), dt.date(2023, 8, 1)]
                  ],
           }

In [324]:
# Choose the path depending if I'm at home or at work
system = os.name
if os.name == "nt":
    path = "D:/" / dc_path
    p3sch_path = "D:/" / dc_p3sch_path
    proposals_path = "D:/" / dc_proposals_path
else:
    path = "/home/fwilde/" / dc_path
    p3sch_path = "/home/fwilde/" / dc_p3sch_path
    proposals_path = "/home/fwilde/" / dc_proposals_path

path

PosixPath('/home/fwilde/desycloud/documents/2023/p05_schedule')

## Import Petra 3 schedule

#### Scripts to filter imported tables and generate empty p05 schedule

In [647]:
##################################################
# Class to work with the p05 schedule
##################################################

class p05sch:
    """
    P05 schedule class.

    Args:
        year <int>: Year for which the p05 schedule should be created
        p3sch_path <str>: path to the PETRA III schedule
        school_holidays <optional, dict>: dictionary containing the school holidays {Bundesland: {Ferienname: daterange}, ...}
        hh_holdays <optional, dict>: dictionary containing the Hamburg holdiays {Feiertagname: date, ...}
    """

    def __init__(self, year, p3sch_path, school_holidays=None, hh_holidays=None, absences=None, proposals_path=None):
        self.year = year
        self.schedule = None
        self.p3_modes = None
        self.p3_addinfo = None
        self.p3_stats = list()
        self.p3sch = pd.read_excel(p3sch_path)
        self.filter()
        self.school_holidays = school_holidays
        self.hh_holidays = hh_holidays
        self.absences = absences

        if not self.schedule:
            self.init()
        if school_holidays:
            self.init_school_holiday_column()

    def filter(self):
        '''
        Strip imported Petra 3 schedule of everything except the Petra III modes and additional info.

        Args:
            p3sch <pandas.DataFrame>:  Petra III schedule read in by read_excel()

        Return:
            p3_modes <pandas.DataFrame>:  table with PETRA III modes
            p3_addinfo <pandas.DataFrame>:  table with additional info
        '''
        counter = 0
        machine_col = []
        addinfo_col = []
        months = ["December", "January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December.1"]
        for col in self.p3sch.iloc[[0]]:
            if col in months:
                machine_col.append(counter + 3)  # 3rd column after the month name column should contain the Petra III modes
                addinfo_col.append(counter + 4)  # 4th column after the month name column should contain the Petra III additional info
            counter += 1
        self.p3_modes = self.p3sch[self.p3sch.columns[machine_col]][1:32]  # fetch mode info from Petra III schedule. Everything beyond row 32 should not belong to the schedule.
        self.p3_addinfo = self.p3sch[self.p3sch.columns[addinfo_col]][1:32]  # fetch additional info from Petra III schedule. Everything beyond row 32 should not belong to the schedule.

        self.p3_modes.columns = months  # set column names to months
        self.p3_addinfo.columns = months  # set column names to months

        self.p3_modes.drop(["December"], axis=1, inplace=True)  # get rid of last years December column
        self.p3_addinfo.drop(["December"], axis=1, inplace=True)  # get rid of last years December column
        self.p3_modes.rename(columns={"December.1": "December"}, inplace=True)  # rename imported December.1 of current year to December
        self.p3_addinfo.rename(columns={"December.1": "December"}, inplace=True)  # rename imported December.1 of current year to December

        stats_col = sch.p3sch.apply(lambda row: row.astype(str).str.contains('per run').any(), axis=1)
        stats_row_index = list(stats_col[stats_col == True].index)
        for entry in sch.p3sch.iloc[stats_row_index[0]]:
            if type(entry) == int:
                self.p3_stats.append(entry)

    def inject_data(self, startdate, enddate, column, data):
        '''
        Injects data into a column of the schedule. Use data=np.nan to remove data from a data frame.

        Args:
            schedule <pandas.DataFrame>
            startdate <datetime.date>
            enddate <datetime.date>
            column <string>
            data <arbitrary>

        Return:
            True
        '''
        dr = pd.date_range(startdate, enddate)
        for date in dr:
            self.schedule.loc[date, column] = data

    def inject_list(self, startdate, enddate, column, data):
        '''
        Injects list data into a column of the schedule. If the list is shorter than the time range,
        missing values will be set to NaN.

        Args:
            startdate <datetime.date>
            enddate <datetime.date>
            column <string>
            data <list> <np.array> <pd.Series>

        Return:
            True
        '''
        dr = pd.date_range(startdate, enddate)  # generate a pandas series wiht dates
        lendiff = len(dr) - len(data)
        if lendiff > 0:  # if the data list is shorter than the list of dates, fill with the missing values with None
            try:
                data.append(None*lendiff)
            except:
                data = np.append(data, [None]*lendiff)

        if type(data) == list:  # in case data of Setup <list>, <np.array>
            for i, date in enumerate(dr):
                self.schedule.loc[date, column] = data[i]
        if type(data) == pd.Series:
            for i, date in enumerate(dr):  # in case data of Setup <pd.Series>
                self.schedule.loc[date, column] = data.iloc[i]

        self.schedule.replace(to_replace=[None], value=np.nan, inplace=True)

    def init(self):
        '''
        Generates an empty p05 schedule which only includes the Petra 3 modes / additional info.

        Args:
            year <int>

        Return:
            schedule <pandas.DataFrame>
        '''
        startdate = dt.date(self.year, 1, 1)
        enddate = dt.date(self.year, 12, 31)
        months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
        schedule_columns = ['School holidays', 'P3 modes', 'P3 info', 'Setup', 'Proposal', 'Application', 'Leader', 'PI', 'Local contact', 'Absence', 'Extra hours', 'Title', 'Conferences', 'Comment']
        self.schedule = pd.DataFrame(index=pd.date_range(startdate, enddate), columns=schedule_columns)
        for month in np.arange(12):
            daysinmonth = calendar.monthrange(year, month+1)[1]  # get number of days in current month
            month_modes = self.p3_modes[months[month]].iloc[:daysinmonth]
            month_addinfo = self.p3_addinfo[months[month]].iloc[:daysinmonth]
            self.inject_list(dt.date(self.year, month+1, 1), dt.date(self.year, month+1, daysinmonth), "P3 modes", month_modes)
            self.inject_list(dt.date(self.year, month+1, 1), dt.date(self.year, month+1, daysinmonth), "P3 info", month_addinfo)
        if self.school_holidays:
            self.init_school_holiday_column()

    def init_school_holiday_column(self):
        '''
        Add school holidays to schedule.  Needs a self.school_holday dictionary to work.

        Args:
            None
        '''
        startdate = dt.date(self.year, 1, 1)
        enddate = dt.date(self.year, 12, 31)
        sh_col = pd.DataFrame(index=pd.date_range(startdate, enddate), columns=["School holidays"])  # grab school holidays column
        for day, holiday in sh_col.iterrows():  # 1. iterate over all days in the year, to compare each day with the school holidays
            holidays_in = ""  # this is the string with the bundesland holidays on a specific days (like "HNS")
            for bundesland, b_school_holidays in self.school_holidays.items():  # 2. iterate over bundesländer
                for holidayname, daterange in b_school_holidays.items():  # within each bundesland iterate over one specific holiday
                    if day in daterange:
                        holidays_in += bundesland[0]
            holidays_string_dict = {"": "", "HNS": "HNS", "H": 'H  ', "N": " N ", "S": "  S", "HN": "HN ", "HS": "H S", "NS": " NS"}
            self.schedule.at[day, "School holidays"] = holidays_string_dict[holidays_in]

    def add_absences(self):
        pass

    def eval_proposals(self, proposals_instance, clear=True):
        if clear:
            self.init()  # Clear entire table
        for appid in proposals_instance.table.index:
            if not pd.isnull(proposals_instance.table["Start date"][appid]):
                days = int(proposals_instance.table["Shifts final"][appid] / 3)
                start_date = proposals_instance.table["Start date"][appid]
                dayspan, i = 0, 0
                while i < days:  # if service days or the like are in the date range, extend the date range
                    currdate = start_date+dt.timedelta(days=dayspan)
                    dayspan += 1
                    if self.schedule.loc[currdate, "P3 modes"] in ["multi", 40]:
                        i += 1

                for day in range(dayspan):
                    currdate = start_date+dt.timedelta(days=day)
                    currapp = self.schedule.loc[currdate, "Application"]
                    if not pd.isna(currapp):  # Check if there is already an application on this date
                        print("{}: Collision for Applications {} and {}".format(currdate, int(appid), int(currapp)))
                    if self.schedule.loc[currdate, "P3 modes"] in ["multi", 40]:
                        for col in ["Setup", "Proposal", "Application", "Leader", "PI", "Local contact", "Title", "Comment"]:
                            if col == "Application":
                                self.schedule.loc[currdate, col] = appid
                            else:
                                self.schedule.loc[currdate, col] = proposals_instance.table.loc[appid, col]

    def _color_index(self, date):
        if date.weekday() == 5:
            color = 'blue'
        elif date.weekday() == 6:
            color = 'mediumblue'
        elif date.date() in list(hh_holidays.values()):
            color = 'orange'
        else:
            color = None

        return 'background-color: {}'.format(color)

    def _color_holidays(self, date):
        if date.date() in list(hh_holidays.values()):
            color = 'orange'
        else:
            color = None
        return 'background-color: %s' % color

    def _color_p3modes(self, mode):
        if mode in ['multi', 40]:
            color = 'seagreen'
        elif mode in ['tr']:
            color = 'green'
        else:
            color = 'maroon'
        return 'background-color: %s' % color


    def _colorize_schedule(self, styler):
        styler.set_caption('P05 schedule %s' % year)
        styler.format(na_rep='')  # don't print NaNs
        styler.applymap_index(self._color_index)
        styler.applymap(self._color_p3modes, subset='P3 modes')
        styler.format(precision=0, subset='Application', na_rep='')
        styler.format('{:3s}', subset='School holidays', na_rep='')
        styler.set_sticky(axis="columns")
        styler.format_index("{:%Y-%m-%d}")  # Remove the time from the date which is introduced by styler
        return styler

    def show(self, startdate=None, enddate=None):
        # self.schedule.loc[dt.datetime(2023, 2, 10):dt.datetime(2023, 8, 1)].style.pipe(colorize_schedule)
        color_schedule = self.schedule.loc[startdate:enddate].style.pipe(self._colorize_schedule)
        display(color_schedule)


##################################################
# Functions to work with table of proposals
##################################################
class proposals:
    """
    The proposals class can read, filter and modify a table with all P05 proposals.

    Args:
        proposals_path <str>: Path to the scheduling support table (list of proposals).
    """

    def __init__(self, proposals_path=None, year=None):
        self.unfiltered_table = pd.read_excel(proposals_path)
        self.table = None
        self.year = year

    def filter(self, filter_string='auto', include=[]):
        '''
        Strip scheduling support table (list of proposal) from all but the current proposals.

        Args:
            filter_string <str>: String used to filter scheduling support table in the "Ranking" column.
                                 If set to "auto", the table is filtered based on the
                                 highest sum of the call and year integers in the
                                 "Ranking" column (eg. "2nd call 2022" -> 2024).
            include <list>: Add a list of Application IDs, that should also be included in the filtered_proposal_table
        '''
        if filter_string == "auto":
            mask_petra = self.unfiltered_table["Ranking"].str.contains("PETRA")  # create mask based on the string "PETRA"
            mask_petra.fillna(False, inplace=True)  # Replace NaN with False in mask
            filtered_proposals = self.unfiltered_table[mask_petra]  # filter proposals based on mask_petra
            r_filtered_proposals = filtered_proposals.reset_index(drop=True)  # rebuild index from 0.. in filtered_proposals
            numbers_in_ranking = filtered_proposals["Ranking"].str.findall(r'([0-9]+)')  # new dataframe generating a list with all numbers (as strings) in the "Ranking" column
            split_numbers_in_ranking = pd.DataFrame(numbers_in_ranking.to_list(), columns=['ranking', 'call', 'year'])  # new dataframe based o
            split_numbers_in_ranking["sum_call_year"] = split_numbers_in_ranking["call"].astype("int") + split_numbers_in_ranking["year"].astype("int")
            max_nir = max(split_numbers_in_ranking["sum_call_year"])
            split_numbers_in_ranking["mask"] = (split_numbers_in_ranking["sum_call_year"] == max_nir)
            mask_newest = split_numbers_in_ranking["mask"]
            self.table = r_filtered_proposals[mask_newest]
        else:
            mask_petra = proposals["Ranking"].str.contains(filter_string)  # create mask based on the filter_string
            mask_petra.fillna(False, inplace=True)  # Replace NaN with False in mask
            filtered_proposals = proposals[mask_petra]  # filter proposals based on mask_petra
            self.table = filtered_proposals.reset_index(drop=True)  # rebuild index from 0.. in filtered_proposals
        include_proposals = self.unfiltered_table[self.unfiltered_table["Application"].isin(include)].copy()
        include_proposals.loc[:, "Ranking (numbers)"] = -1
        # Add column sthat will be needed later on
        self.table = pd.concat([self.table, include_proposals])
        self.table.set_index("Application", inplace=True)  # prevent multiple identical Application IDs
        self.table.insert(8, "Shifts final", None)
        self.table.insert(9, "Start date", pd.NaT)
        self.table.insert(19, "Unacceptable list", pd.NaT)
        self.table.insert(10, "Local contact", None)
        self.table["Comment"] = None

    def convert_unacceptables(self, verbose=False):
        months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
        for appid, unacceptable in self.table["Unacceptable"].items():
            if verbose:
                print("Leader/ PI / beamtime: {} / {} / {}".format(self.table.loc[appid, "Leader"], self.table.loc[appid, "PI"], appid))
                print("user input: \"{}\"".format(unacceptable))
            if pd.isnull(unacceptable):
                pass
            else:
                split_separator = re.split(r'[,;]+', unacceptable)
                split_range_symbol = None
                unacceptable_list = []
                # Try first to extract single month names and generate a daterange from that
                for dt_range in split_separator:
                    fz_result = [fz.fuzz.partial_ratio(item, dt_range) for item in months]
                    try:
                        match = fz_result.index(100) + 1
                        if match:
                            daysinmonth = calendar.monthrange(year, match)[1]  # get number of days in current month
                            startdate = "{}-{}-{}".format(year, match, 1)
                            enddate = "{}-{}-{}".format(year, match, daysinmonth)
                            unacceptable_list.append(pd.date_range(start=startdate, end=enddate))
                            continue
                    except:
                        # then go ahead and try to convert dates directly
                        split_separator = dt_range.replace(" to ", "-")
                        split_range_symbol = re.split(r'[-]+', split_separator)
                        contains_dot = ["." in item for item in split_range_symbol]
                        if True in contains_dot:
                            startdate = [item for item in split_range_symbol[0].split(".") if item]  # split date by . and make sure that there are no empty strings
                            enddate = [item for item in split_range_symbol[1].split(".") if item]  # split date by . and make sure that there are no empty strings
                            startday = startdate[0].replace(" ", "")
                            endday = enddate[0].replace(" ", "")
                            if len(enddate) > 1:
                                endmonth = enddate[1]
                            if len(startdate) > 1:
                                startmonth = startdate[1].replace(" ", "")
                            else:
                                startmonth = endmonth
                            unacceptable_list.append(pd.date_range(start="{}-{}-{}".format(year, startmonth, startday), end="{}-{}-{}".format(year, endmonth, endday)))
                            self.table.at[appid, "Unacceptable list"] = unacceptable_list
                        else:  # give up if that fails
                            print("Warning: Application {} ({} / {}): Couldn't convert \"{}\" to datetime.".format(appid, self.table.loc[appid, "Leader"], self.table.loc[appid, "PI"], unacceptable))

                if unacceptable_list:  # flatten unacceptable list
                    unacceptable_list = pd.DatetimeIndex([item for sublist in unacceptable_list for item in sublist])
                    self.table.at[appid, "Unacceptable list"] = unacceptable_list
                    if verbose:
                        print("List of unaccepatable dates: \n{}".format(unacceptable_list))
            if verbose:
                print("=====================================================")

    def modify_unacceptables(self, appid, entry):
        self.table.at[appid, "Unacceptable"] = entry

    def show(self, exclude=['Collaboration', 'Ranking', 'Beamline', 'Submitted', 'Filling mode (bunches)', 'Beam size'], include=[]):
        """
        Show the filtered proposal table and optionally exclude columns. If an include list is provided only
        the columns in the inlude list will be displayed.

        Args:
            exclude <str list>: Optional. List of columns that should be excluded in display. Defaults to
                                ['Collaboration','Ranking', 'Beamline','Submitted','Filling mode (bunches)','Beam size']
            include <str list>: Optional. List of columns that shoud be included in display.  Defauilts to [].
        """
        if not include:
            display(self.table.loc[:, ~self.table.columns.isin(exclude)].sort_values(by=["Ranking (numbers)", "Proposal"]))
        else:
            display(self.table.loc[:, self.table.columns.isin(include)])

    def assign(self, appid, shifts=9, local_contact=None, start_date=None, comment=None):
        """
        Assign a local contact, number of shifts, Start date and a comment to an application ID and enter these values in the proposal table.

        Args:
            appid <int>: Application ID
            number_of_shifts <optional, int>: How many shifts should be assigned to a beamtime, defaults to 9 (!)
            local_contact <optional, str>: Local contact (initials)
            start_date <optional, dt.date>: When shoudl the beamtime begin
            comment <optional, str>: A comment tha should go along with the beamtime
        """
        if shifts:
            self.table.at[appid, "Shifts final"] = shifts
        else:
            self.table.at[appid, "Shifts final"] = self.table.loc[appid, "Shifts assigned"]
        if local_contact:
            self.table.at[appid, "Local contact"] = local_contact
        if start_date:
            self.table.at[appid, "Start date"] = start_date
            beamtime_dates = [(start_date+dt.timedelta(days=i)).strftime("%Y-%m-%d") for i in range(int(shifts/3))]
            if self._check_unacceptables(appid, beamtime_dates):
                print("Warning: Beamtime {} ({} / {}): startdate {} lies with the unacceptable dates.".format(appid, self.table.loc[appid, "Leader"], self.table.loc[appid, "PI"], start_date.strftime("%Y-%m-%d")))
        if comment:
            self.table.at[appid, "Comment"] = comment

    def add_beamtime(self, appid, proposal=None, setup=None, title=None, leader=None, pi=None, ranking=100, shifts=None, start_date=None, comment=None):
        """
        Add a new beamtime (eg. commissioning or inhouse).
        """
        setup_dict = {'micro': 'Microtomography (EH2)', 'nano': 'Nanotomography (EH1)'}
        self.table.loc[appid] = {'Proposal': proposal, 'Setup': setup_dict[setup], 'Ranking (numbers)': ranking, 'Title': title, 'Leader': leader, 'PI': pi, 'Shifts assigned': shifts, 'Shifts final': shifts, 'Start date': start_date, "Comment": comment}

    def print_assign_commands(self, instance_name):
        print_table = self.table[["Setup", "Ranking (numbers)", "Leader", "PI", "Shifts assigned"]].sort_values(by=["Setup", "Ranking (numbers)"])
        #display(print_table)
        for appid, setup in print_table["Setup"].items():
            print("#  {}.assign({}, local_contact=\"XX\", start_date=dt.datetime({}, 1, 1), shifts={})  # {} / {}, {}".format(instance_name, appid, self.year, print_table.loc[appid, "Shifts assigned"], print_table.loc[appid, "Leader"], print_table.loc[appid, "PI"], print_table.loc[appid, "Setup"]))

    def _check_unacceptables(self, appid, date_list):
        conflict_list = list()
        for date in date_list:
            if type(pd.isna(self.table.loc[appid, "Unacceptable list"])) is not bool:
                conflict_list.append(date in self.table.loc[appid, "Unacceptable list"])
        if conflict_list:
            return all(conflict_list)

### Initialize schedule and proposal_table

In [648]:
sch = p05sch(2023, p3sch_path, school_holidays=school_holidays, hh_holidays=hh_holidays)
sch.init_school_holiday_column()

In [649]:
# This part only works, if the scheduling support (list of proposals) is available
# The order of execution is important.

# proposals = filter_proposals(proposals, filter_string="2nd call 2022")
pro = proposals(proposals_path, year)

# Add applications that are also in the unfiltered proposal table
pro.filter(include=[11016379, 11016663, 11016664, 11016665, 11016666, 11016667, 11016668])

# Add commissioning, inhouse, industry etc
pro.add_beamtime(101, proposal='micro comm', setup='micro', title='micro commissioning', shifts=6)
pro.add_beamtime(102, proposal='micro comm', setup='micro', title='micro commissioning', shifts=6)
pro.add_beamtime(103, proposal='micro comm', setup='micro', title='micro commissioning', shifts=3)
pro.add_beamtime(104, proposal='micro comm', setup='micro', title='micro commissioning', shifts=3)

pro.add_beamtime(201, proposal='nano comm', setup='nano', title='nano commissioning', shifts=6)
pro.add_beamtime(11013127, proposal='I-20211523 EC', setup='micro', leader="Sket", pi="Sket", title='Grain boundary study in hexagonal microstructures by X-ray radiography and tomography during directional solidification', shifts=9, ranking=-1) 

# Modify unaceptables
pro.modify_unacceptables(11016412, None)
pro.modify_unacceptables(11016352, '01.02-15.03')
pro.modify_unacceptables(11016060, 'january, february, march')
pro.modify_unacceptables(11015941, 'january, february, march')
pro.modify_unacceptables(11015918, '01.05-31.05')
pro.modify_unacceptables(11015832, '09.01-20.01')
pro.convert_unacceptables(verbose=False)

# This next command is only needed once to prepare the assignment of proposals
# Copy the output of this command below
# pro.print_assign_commands(instance_name="pro")

# Assign local contact, start date, shifts and comment to appilcations

#  pro.assign(11016668, local_contact="JH", start_date=dt.datetime(2023, 1, 1), shifts=72)  # Wipfler / Hammel, Microtomography (EH2)
#  pro.assign(11016667, local_contact="JH", start_date=dt.datetime(2023, 1, 1), shifts=72)  # Wipfler / Hammel, Microtomography (EH2)
pro.assign(11016666, local_contact="JH", start_date=dt.datetime(2023, 2, 27), shifts=9)  # Herzen / Hammel, Microtomography (EH2)
pro.assign(11016665, local_contact="JH", start_date=dt.datetime(2023, 2, 24), shifts=9)  # Herzen / Hammel, Microtomography (EH2)
#  pro.assign(11016664, local_contact="JH", start_date=dt.datetime(2023, 1, 1), shifts=72)  # Wipfler / Hammel, Microtomography (EH2)
#  pro.assign(11016663, local_contact="JH", start_date=dt.datetime(2023, 1, 1), shifts=72)  # Wipfler / Hammel, Microtomography (EH2)
#  pro.assign(11016379, local_contact="FW", start_date=dt.datetime(2023, 1, 1), shifts=6)  # Rout / Rout, Microtomography (EH2)
pro.assign(11013127, local_contact="FW", start_date=dt.datetime(2023, 7, 7), shifts=9)  # Sket / Sket, Microtomography (EH2)
#  pro.assign(11016060, local_contact="JH", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Hammel / Naumann, Microtomography (EH2)
#  pro.assign(11016358, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=6)  # Blawert / Fazel, Microtomography (EH2)
#  pro.assign(11016384, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=12)  # Wieland / Hindenlang, Microtomography (EH2)
#  pro.assign(11015941, local_contact="FW", start_date=dt.datetime(2023, 1, 1), shifts=12)  # Manke / SUN, Microtomography (EH2)
#  pro.assign(11016078, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=15)  # Adelung / Zeller-Plumhoff, Microtomography (EH2)
#  pro.assign(11015864, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Mayer / Mayer, Microtomography (EH2)
pro.assign(11016439, local_contact="FW", start_date=dt.datetime(2023, 3, 3), shifts=9)  # Ziesche / Ziesche, Microtomography (EH2)
#  pro.assign(11016279, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=6)  # Dultz / Dultz, Microtomography (EH2)
pro.assign(11016144, local_contact="FW", start_date=dt.datetime(2023, 7, 14), shifts=9)  # Pistidda / Karimi, Microtomography (EH2)
#  pro.assign(11015832, local_contact="FW", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Manke / Osenberg, Microtomography (EH2)
#  pro.assign(11016289, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Engqvist / Engqvist, Microtomography (EH2)
#  pro.assign(11016351, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=3)  # Hejnol / Hejnol, Microtomography (EH2)
#  pro.assign(11015942, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=6)  # Blasco-Costa / Blasco-Costa, Microtomography (EH2)
#  pro.assign(11015918, local_contact="FW", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Manke / Paulisch, Microtomography (EH2)
#  pro.assign(11016301, local_contact="FW", start_date=dt.datetime(2023, 1, 1), shifts=12)  # Manke / Arlt, Microtomography (EH2)
#  pro.assign(11016415, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Engelkes / Busse, Microtomography (EH2)
pro.assign(101, local_contact="", start_date=dt.datetime(2023, 2, 22), shifts=6)  # nan / nan, Microtomography (EH2)
pro.assign(102, local_contact="", start_date=dt.datetime(2023, 3, 6), shifts=6)  # nan / nan, Microtomography (EH2)
pro.assign(103, local_contact="", start_date=dt.datetime(2023, 7, 6), shifts=3)  # nan / nan, Microtomography (EH2)
pro.assign(104, local_contact="", start_date=dt.datetime(2023, 7, 13), shifts=3)  # nan / nan, Microtomography (EH2)


#  pro.assign(11016412, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=12)  # Hesse / Hesse, Nanotomography (EH1)
#  pro.assign(11016342, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Zippo / Zippo, Nanotomography (EH1)
#  pro.assign(11015833, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Konchakova / Konchakova, Nanotomography (EH1)
#  pro.assign(11016480, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=12)  # Greving / Herzen, Nanotomography (EH1)
#  pro.assign(11016186, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Tolnai / Tolnai, Nanotomography (EH1)
#  pro.assign(11015933, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=12)  # Ni / Ni, Nanotomography (EH1)
#  pro.assign(11016352, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=3)  # Oliveira / Oliveira, Nanotomography (EH1)
#  pro.assign(11016103, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Zeng / Xu, Nanotomography (EH1)
#  pro.assign(11016148, local_contact="XX", start_date=dt.datetime(2023, 1, 1), shifts=9)  # Soederberg / Gordeyeva, Nanotomography (EH1)
pro.assign(201, local_contact="", start_date=dt.datetime(2023, 3, 9), shifts=6)  # nan / nan, Microtomography (EH2)


pro.show()

,Proposal,Setup,Ranking (numbers),Shifts applied,Shifts assigned,Shifts final,Start date,Local contact,Title,Leader,PI,Energy,Preferred dates,Unacceptable,Unacceptable list,Comment
Application,,,,,,,,,,,,,,,,
11016664,BAG-20210019,Microtomography (EH2),-1,6.0,72,None,NaN,None,Understanding different scales oft the ecosyst...,Wipfler,Hammel,18 keV,NaN,NaN,NaN,None
11016663,BAG-20210019,Microtomography (EH2),-1,12.0,72,None,NaN,None,Understanding different scales oft the ecosyst...,Wipfler,Hammel,18 keV,NaN,NaN,NaN,None
11016666,BAG-20211054,Microtomography (EH2),-1,6.0,72,9,2023-02-27 00:00:00,JH,High-sensitivity quantitative phase-contrast t...,Herzen,Hammel,30 keV,NaN,NaN,NaN,None
11016665,BAG-20211054,Microtomography (EH2),-1,12.0,72,9,2023-02-24 00:00:00,JH,High-sensitivity quantitative phase-contrast t...,Herzen,Hammel,30 keV,NaN,NaN,NaN,None
11016668,BAG-20211055,Microtomography (EH2),-1,6.0,72,None,NaN,None,The arthropod tracheal system in the context o...,Wipfler,Hammel,30 keV,NaN,NaN,NaN,None
11016667,BAG-20211055,Microtomography (EH2),-1,12.0,72,None,NaN,None,The arthropod tracheal system in the context o...,Wipfler,Hammel,25 keV,NaN,NaN,NaN,None
11013127,I-20211523 EC,Microtomography (EH2),-1,NaN,9,9,2023-07-07 00:00:00,FW,Grain boundary study in hexagonal microstructu...,Sket,Sket,NaN,NaN,NaN,NaN,None
11016379,I-20221317,Microtomography (EH2),-1,6.0,6,None,NaN,None,Chromite grains in Antarctic micrometeorites: ...,Rout,Rout,18 keV,NaN,NaN,NaN,None
11016060,I-20221037,Microtomography (EH2),1,9.0,9,None,NaN,None,In vivo imaging of the functional morphology o...,Hammel,Naumann,30-33,May to July,"january, february, march","[2023-01-01 00:00:00, 2023-01-02 00:00:00, 202...",None


In [604]:
# preassign nano and micro
sch.init()
sch.inject_data(dt.date(2023,3,9), dt.date(2023,4,2), "Setup", "nano")
sch.inject_data(dt.date(2023,6,1), dt.date(2023,6,20), "Setup", "nano")

sch.eval_proposals(pro, clear=False)
sch.show(startdate=dt.datetime(2023, 2, 10), enddate=dt.datetime(2023, 8, 1))

,School holidays,P3 modes,P3 info,Setup,Proposal,Application,Leader,PI,Local contact,Absence,Extra hours,Title,Conferences,Comment
2023-02-10,,,IB,,,,,,,,,,,
2023-02-11,,,IB (tr),,,,,,,,,,,
2023-02-12,,,IB (tr),,,,,,,,,,,
2023-02-13,,,IB,,,,,,,,,,,
2023-02-14,,,IB,,,,,,,,,,,
2023-02-15,,A,IB tr,,,,,,,,,,,
2023-02-16,,,IB tr,,,,,,,,,,,
2023-02-17,,,IB tr,,,,,,,,,,,
2023-02-18,,tr,,,,,,,,,,,,
2023-02-19,,tr,,,,,,,,,,,,


# Write tables to excel file

In [591]:
if 'Unacceptable list' in pro.table:
    pro.table.drop("Unacceptable list", axis=1, inplace=True)
with pd.ExcelWriter("/home/fwilde/tmp/p05_schedule_{}.xlsx".format(year), date_format='YYYY-MM-DD', datetime_format='YYYY-MM-DD') as writer:
    csch.to_excel(writer, sheet_name="schedule", index=True, freeze_panes=(1,3))
    pro.table.to_excel(writer, sheet_name="proposals_1", index=True, freeze_panes=(1,1))

# Test zone

In [520]:
r = pd.date_range(dt.datetime(2023,1,1), dt.datetime(2023,2,1))
len(r)

32

In [650]:
sch.p3_stats

[109, 99, 208]